# 학습 전

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/000GithubRepos/000recsys_key_papers_implementation/model_comparison/kyeongchan/NGCF')

In [ ]:
pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from NGCF import NGCF
from load_data import Data

In [27]:
# This file is based on the NGCF author's implementation
# <https://github.com/xiangwang1223/neural_graph_collaborative_filtering/blob/master/NGCF/utility/parser.py>.

import argparse


def parse_args():
    parser = argparse.ArgumentParser(description="Run NGCF.")
    parser.add_argument("--weights_path", nargs="?", default="model/", help="Store model path.")
    parser.add_argument("--data_path", nargs="?", default="../Data/", help="Input data path.")
    parser.add_argument("--model_name", type=str, default="NGCF.pkl", help="Saved model name.")
    parser.add_argument("--dataset",nargs="?",default="gowalla",help="Choose a dataset from {gowalla, yelp2018, amazon-book}",)
    parser.add_argument("--verbose", type=int, default=1, help="Interval of evaluation.")
    parser.add_argument("--epoch", type=int, default=400, help="Number of epoch.")

    parser.add_argument("--embed_size", type=int, default=64, help="Embedding size.")
    parser.add_argument("--layer_size",nargs="?",default="[64,64,64]",help="Output sizes of every layer",)
    parser.add_argument("--batch_size", type=int, default=1024, help="Batch size.")
    parser.add_argument("--regs", nargs="?", default="[1e-5]", help="Regularizations.")
    parser.add_argument("--lr", type=float, default=0.0001, help="Learning rate.")
    parser.add_argument("--gpu", type=int, default=0, help="0 for NAIS_prod, 1 for NAIS_concat")
    parser.add_argument("--mess_dropout",nargs="?",default="[0.1,0.1,0.1]",help="Keep probability w.r.t. message dropout (i.e., 1-dropout_ratio) for each deep layer. 1: no dropout.",)
    parser.add_argument("--Ks",nargs="?",default="[20, 40]",help="Output sizes of every layer",)

    parser.add_argument("--save_flag",type=int,default=1,help="0: Disable model saver, 1: Activate model saver",)
    parser.add_argument("--test_flag",nargs="?",default="part",help="Specify the test type from {part, full}, indicating whether the reference is done in mini-batch",)

    parser.add_argument("--report",type=int,default=0,help="0: Disable performance report w.r.t. sparsity levels, 1: Show performance report w.r.t. sparsity levels",)
    return parser.parse_args([])

args = parse_args()
Ks = eval(args.Ks)
vars(args)

{'weights_path': 'model/',
 'data_path': '../Data/',
 'model_name': 'NGCF.pkl',
 'dataset': 'gowalla',
 'verbose': 1,
 'epoch': 400,
 'embed_size': 64,
 'layer_size': '[64,64,64]',
 'batch_size': 1024,
 'regs': '[1e-5]',
 'lr': 0.0001,
 'gpu': 0,
 'mess_dropout': '[0.1,0.1,0.1]',
 'Ks': '[20, 40]',
 'save_flag': 1,
 'test_flag': 'part',
 'report': 0}

In [28]:
args.data_path = os.path.join('Data/')

In [29]:
args.batch_size = 128

## batch_test import * 하기

In [ ]:
import heapq
import multiprocessing

# import utility.metrics as metrics
# from utility.load_data import *
# from utility.parser import parse_args

cores = multiprocessing.cpu_count()

data_generator = Data(path=args.data_path + args.dataset, batch_size=args.batch_size)
USR_NUM, ITEM_NUM = data_generator.n_users, data_generator.n_items
N_TRAIN, N_TEST = data_generator.n_train, data_generator.n_test
BATCH_SIZE = args.batch_size

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084


In [ ]:
data_generator = Data(path=args.data_path + args.dataset, batch_size=args.batch_size)

n_users=29858, n_items=40981
n_interactions=1027370
n_train=810128, n_test=217242, sparsity=0.00084


In [11]:
USR_NUM, ITEM_NUM = data_generator.n_users, data_generator.n_items
N_TRAIN, N_TEST = data_generator.n_train, data_generator.n_test
BATCH_SIZE = args.batch_size

## main

In [12]:
args.weights_path

'model/'

In [13]:
if not os.path.exists(args.weights_path):
    os.mkdir(args.weights_path)

In [14]:
args.mess_dropout = eval(args.mess_dropout)
args.mess_dropout
args.layer_size = eval(args.layer_size)
args.regs = eval(args.regs)
vars(args)

{'weights_path': 'model/',
 'data_path': 'Data/',
 'model_name': 'NGCF.pkl',
 'dataset': 'gowalla',
 'verbose': 1,
 'epoch': 400,
 'embed_size': 64,
 'layer_size': [64, 64, 64],
 'batch_size': 128,
 'regs': [1e-05],
 'lr': 0.0001,
 'gpu': 0,
 'mess_dropout': [0.1, 0.1, 0.1],
 'Ks': '[20, 40]',
 'save_flag': 1,
 'test_flag': 'part',
 'report': 0}

In [15]:
import os
from time import time

import torch
import torch.optim as optim

In [16]:
args.gpu

0

In [17]:
if args.gpu >= 0 and torch.cuda.is_available():
    device = "cuda:{}".format(args.gpu)
else:
    device = "cpu"

In [18]:
g = data_generator.g
g = g.to(device)

## Step 2: Create model and training components=========================================================== #


In [19]:
model = NGCF(g, args.embed_size, args.layer_size, args.mess_dropout, args.regs[0]).to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

In [20]:
model

NGCF(
  (layers): ModuleList(
    (0-2): 3 x NGCFLayer(
      (W1): Linear(in_features=64, out_features=64, bias=True)
      (W2): Linear(in_features=64, out_features=64, bias=True)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (feature_dict): ParameterDict(
      (item): Parameter containing: [torch.FloatTensor of size 40981x64]
      (user): Parameter containing: [torch.FloatTensor of size 29858x64]
  )
)

## Step 3: training epoches ============================================================================== #

In [31]:
n_batch = data_generator.n_train // args.batch_size + 1
t0 = time()
cur_best_pre_0, stopping_step = 0, 0
loss_loger, pre_loger, rec_loger, ndcg_loger, hit_loger = [], [], [], [], []

# 학습

## 1epoch 학습

In [ ]:
model.train()

NGCF(
  (layers): ModuleList(
    (0-2): 3 x NGCFLayer(
      (W1): Linear(in_features=64, out_features=64, bias=True)
      (W2): Linear(in_features=64, out_features=64, bias=True)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (feature_dict): ParameterDict(
      (item): Parameter containing: [torch.FloatTensor of size 40981x64]
      (user): Parameter containing: [torch.FloatTensor of size 29858x64]
  )
)

In [24]:
perf_str = "Epoch %d [%.1fs]: train==[%.5f=%.5f + %.5f]" % (
    epoch,
    time() - t1,
    loss,
    mf_loss,
    emb_loss,
)
print(perf_str)

NameError: ignored

In [ ]:
type(ret)

tuple

In [35]:
import numpy as np

In [25]:
from batch_test import test


In [42]:
result = {
    "precision": np.zeros(len(Ks)),
    "recall": np.zeros(len(Ks)),
    "ndcg": np.zeros(len(Ks)),
    "hit_ratio": np.zeros(len(Ks)),
    "auc": 0.0,
}

pool = multiprocessing.Pool(cores)

u_batch_size = 5000
i_batch_size = BATCH_SIZE

test_users = users_to_test
n_test_users = len(test_users)
n_user_batchs = n_test_users // u_batch_size + 1

count = 0

In [43]:
batch_test_flag = False
for u_batch_id in range(n_user_batchs):
    start = u_batch_id * u_batch_size
    end = (u_batch_id + 1) * u_batch_size

    user_batch = test_users[start:end]

    if batch_test_flag:
        # batch-item test
        n_item_batchs = ITEM_NUM // i_batch_size + 1
        rate_batch = np.zeros(shape=(len(user_batch), ITEM_NUM))

        i_count = 0
        for i_batch_id in range(n_item_batchs):
            i_start = i_batch_id * i_batch_size
            i_end = min((i_batch_id + 1) * i_batch_size, ITEM_NUM)

            item_batch = range(i_start, i_end)

            u_g_embeddings, pos_i_g_embeddings, _ = model(
                g, "user", "item", user_batch, item_batch, []
            )
            i_rate_batch = (
                model.rating(u_g_embeddings, pos_i_g_embeddings)
                .detach()
                .cpu()
            )

            rate_batch[:, i_start:i_end] = i_rate_batch
            i_count += i_rate_batch.shape[1]

        assert i_count == ITEM_NUM

    else:
        # all-item test
        item_batch = range(ITEM_NUM)
        u_g_embeddings, pos_i_g_embeddings, _ = model(
            g, "user", "item", user_batch, item_batch, []
        )
        rate_batch = (
            model.rating(u_g_embeddings, pos_i_g_embeddings).detach().cpu()
        )

    user_batch_rating_uid = zip(rate_batch.numpy(), user_batch)
    batch_result = pool.map(test_one_user, user_batch_rating_uid)
    count += len(batch_result)

    for re in batch_result:
        result["precision"] += re["precision"] / n_test_users
        result["recall"] += re["recall"] / n_test_users
        result["ndcg"] += re["ndcg"] / n_test_users
        result["hit_ratio"] += re["hit_ratio"] / n_test_users
        result["auc"] += re["auc"] / n_test_users

NameError: ignored

In [39]:
def test_one_user(x):
    # user u's ratings for user u
    rating = x[0]
    # uid
    u = x[1]
    # user u's items in the training set
    try:
        training_items = data_generator.train_items[u]
    except Exception:
        training_items = []
    # user u's items in the test set
    user_pos_test = data_generator.test_set[u]

    all_items = set(range(ITEM_NUM))

    test_items = list(all_items - set(training_items))

    if args.test_flag == "part":
        r, auc = ranklist_by_heapq(user_pos_test, test_items, rating, Ks)
    else:
        r, auc = ranklist_by_sorted(user_pos_test, test_items, rating, Ks)

    return get_performance(user_pos_test, r, auc, Ks)

In [40]:
def ranklist_by_heapq(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.0
    return r, auc


def get_auc(item_score, user_pos_test):
    item_score = sorted(item_score.items(), key=lambda kv: kv[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]
    posterior = [x[1] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = metrics.auc(ground_truth=r, prediction=posterior)
    return auc


def ranklist_by_sorted(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = get_auc(item_score, user_pos_test)
    return r, auc


def get_performance(user_pos_test, r, auc, Ks):
    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(metrics.precision_at_k(r, K))
        recall.append(metrics.recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(metrics.ndcg_at_k(r, K))
        hit_ratio.append(metrics.hit_at_k(r, K))

    return {
        "recall": np.array(recall),
        "precision": np.array(precision),
        "ndcg": np.array(ndcg),
        "hit_ratio": np.array(hit_ratio),
        "auc": auc,
    }

In [32]:
for epoch in range(args.epoch):
    t1 = time()
    loss, mf_loss, emb_loss = 0.0, 0.0, 0.0
    for idx in range(n_batch):
        users, pos_items, neg_items = data_generator.sample()
        u_g_embeddings, pos_i_g_embeddings, neg_i_g_embeddings = model(
            g, "user", "item", users, pos_items, neg_items
        )

        batch_loss, batch_mf_loss, batch_emb_loss = model.create_bpr_loss(
            u_g_embeddings, pos_i_g_embeddings, neg_i_g_embeddings
        )
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss
        mf_loss += batch_mf_loss
        emb_loss += batch_emb_loss

    if (epoch + 1) % 10 != 0:
        if args.verbose > 0 and epoch % args.verbose == 0:
            perf_str = "Epoch %d [%.1fs]: train==[%.5f=%.5f + %.5f]" % (
                epoch,
                time() - t1,
                loss,
                mf_loss,
                emb_loss,
            )
            print(perf_str)
        continue  # end the current epoch and move to the next epoch, let the following evaluation run every 10 epoches

    # evaluate the model every 10 epoches
    t2 = time()
    users_to_test = list(data_generator.test_set.keys())
    ret = test(model, g, users_to_test)
    t3 = time()

    loss_loger.append(loss)
    rec_loger.append(ret["recall"])
    pre_loger.append(ret["precision"])
    ndcg_loger.append(ret["ndcg"])
    hit_loger.append(ret["hit_ratio"])

    if args.verbose > 0:
        perf_str = (
            "Epoch %d [%.1fs + %.1fs]: train==[%.5f=%.5f + %.5f], recall=[%.5f, %.5f], "
            "precision=[%.5f, %.5f], hit=[%.5f, %.5f], ndcg=[%.5f, %.5f]"
            % (
                epoch,
                t2 - t1,
                t3 - t2,
                loss,
                mf_loss,
                emb_loss,
                ret["recall"][0],
                ret["recall"][-1],
                ret["precision"][0],
                ret["precision"][-1],
                ret["hit_ratio"][0],
                ret["hit_ratio"][-1],
                ret["ndcg"][0],
                ret["ndcg"][-1],
            )
        )
        print(perf_str)

    cur_best_pre_0, stopping_step, should_stop = early_stopping(
        ret["recall"][0],
        cur_best_pre_0,
        stopping_step,
        expected_order="acc",
        flag_step=5,
    )

    # early stop
    if should_stop == True:
        break

    if ret["recall"][0] == cur_best_pre_0 and args.save_flag == 1:
        torch.save(model.state_dict(), args.weights_path + args.model_name)
        print(
            "save the weights in path: ",
            args.weights_path + args.model_name,
        )

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), args.weights_path + args.model_name)

In [ ]:
args.weights_path + args.model_name

'model/NGCF.pkl'

In [ ]:
torch.save(model.state_dict(), args.weights_path + args.model_name)

In [23]:

model.load_state_dict(torch.load(args.weights_path + args.model_name))
model.eval()

NGCF(
  (layers): ModuleList(
    (0-2): 3 x NGCFLayer(
      (W1): Linear(in_features=64, out_features=64, bias=True)
      (W2): Linear(in_features=64, out_features=64, bias=True)
      (leaky_relu): LeakyReLU(negative_slope=0.2)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (feature_dict): ParameterDict(
      (item): Parameter containing: [torch.FloatTensor of size 40981x64]
      (user): Parameter containing: [torch.FloatTensor of size 29858x64]
  )
)

In [21]:
import psutil
split_bar = '='*20
memory_info = psutil.virtual_memory()._asdict()
print(f"{split_bar} Memory Usage {split_bar}")
for k,v in memory_info.items():
  print(k, v)
print(f"{split_bar} CPU Usage {split_bar}")
print(f"CPU percent: {psutil.cpu_percent()}%")

==================== Memory Usage ====================
total 37836288000
available 35352576000
percent 6.6
used 1953996800
free 31852179456
active 972435456
inactive 4266545152
buffers 455294976
cached 3574816768
shared 1945600
slab 344133632
==================== CPU Usage ====================
CPU percent: 2.3%
